In [ ]:
%run ./../template.ipynb

In [2]:
path = kagglehub.dataset_download("rohanrao/air-quality-data-in-india")

print("Path to dataset files:", path)

100%|██████████| 72.9M/72.9M [00:23<00:00, 3.25MB/s]

Extracting files...


Path to dataset files: C:\Users\2814\.cache\kagglehub\datasets\rohanrao\air-quality-data-in-india\versions\12


In [ ]:
data_path = Path(config['ml']['data_folder'])
shutil.move(path, data_path)

In [4]:
train_df = pd.read_csv(data_path / "12/city_day.csv")
val_df = pd.read_csv(data_path / "12/city_day.csv")

In [5]:
train_df.head()
print(train_df.shape)

(29531, 16)


In [6]:
print(val_df.shape)

(29531, 16)


In [ ]:
# Пропущенные поля
print(train_df.isnull().sum())
# Убираем колонки 
train_df.drop(columns=["City", "Date"], inplace=True)
# Заполнение NaN (например, медианой или средним) - надо ли нам это?
# train_df.head().fillna(train_df.head().median(), inplace=True) 

In [24]:
# Здесь мы фильтруем данные для города "Ahmedabad" и выбираем столбцы "Date", "PM2.5", "CO", "O3"
city_data = train_df[train_df["City"] == "Ahmedabad"][["Date", "PM2.5", "CO", "O3"]]

# Теперь заполним пропуски (NaN) в выбранных столбцах медианой этих столбцов
city_data[["PM2.5", "CO", "O3"]] = city_data[["PM2.5", "CO", "O3"]].fillna(city_data[["PM2.5", "CO", "O3"]].median())
print(city_data)

            Date  PM2.5     CO      O3
0     2015-01-01  58.37   0.92  133.36
1     2015-01-02  58.37   0.97   34.06
2     2015-01-03  58.37  17.40   30.70
3     2015-01-04  58.37   1.70   36.08
4     2015-01-05  58.37  22.10   39.31
...          ...    ...    ...     ...
2004  2020-06-27  62.12   0.49   68.05
2005  2020-06-28  31.57   0.52   26.34
2006  2020-06-29  29.75   0.67   34.99
2007  2020-06-30  40.02   0.73   41.64
2008  2020-07-01  37.63   0.28    9.69

[2009 rows x 4 columns]


In [25]:
from transformers import TimeSeriesTransformerForPrediction

model = TimeSeriesTransformerForPrediction.from_pretrained("kleopatra102/air_pollution")

print(model)

TimeSeriesTransformerForPrediction(
  (model): TimeSeriesTransformerModel(
    (scaler): TimeSeriesMeanScaler()
    (embedder): TimeSeriesFeatureEmbedder(
      (embedders): ModuleList(
        (0): Embedding(270, 2)
      )
    )
    (encoder): TimeSeriesTransformerEncoder(
      (value_embedding): TimeSeriesValueEmbedding(
        (value_projection): Linear(in_features=49, out_features=32, bias=False)
      )
      (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(96, 32)
      (layers): ModuleList(
        (0-3): 4 x TimeSeriesTransformerEncoderLayer(
          (self_attn): TimeSeriesTransformerAttention(
            (k_proj): Linear(in_features=32, out_features=32, bias=True)
            (v_proj): Linear(in_features=32, out_features=32, bias=True)
            (q_proj): Linear(in_features=32, out_features=32, bias=True)
            (out_proj): Linear(in_features=32, out_features=32, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((32,), eps=1e-05, element